In [11]:
import numpy as np
import scipy
import scipy.ndimage
import scipy.sparse.linalg
import matplotlib.pyplot as plt

In [ ]:
def make_boolean_grid(x,y,fractal):
    '''
    Makes boolean grid out of fractal
    '''

    grid = np.zeros((len(x),len(y)))

    for i in range(len(fractal)):
        
        grid[np.where(x == fractal[i,0]), np.where(y == fractal[i,1])] = 1


    return grid

In [ ]:
def flatten_fractal(fractal):
    '''
    Flattens the fractal, so that we can make the sparse matrix
    '''
    
    inside_fractal_with_border = scipy.ndimage.binary_fill_holes(fractal).astype(int)
    
    inside_fractal = inside_fractal_with_border-fractal

    plt.imshow(inside_fractal)
    plt.show()
    inside_fractal_flat = inside_fractal.flatten()
    
    return inside_fractal, inside_fractal_flat

def flatten_fractal_task8(booleanGrid):
    
    inside_fractal_with_border = scipy.ndimage.binary_fill_holes(booleanGrid).astype(int)
    
    inside_fractal = inside_fractal_with_border-booleanGrid

    #Making modification for the new boundary condition
    for i in range(inside_fractal.shape[0]):
        for j in range(inside_fractal.shape[1]):

            if inside_fractal[i,j] == 1:
            
            # Checking the borders
                number = 1
            
                if inside_fractal[i,j+1] == 0:
                    number += 1

                if inside_fractal[i,j-1] == 0:
                    number += 1

                if inside_fractal[i+1,j] == 0:
                    number += 1

                if inside_fractal[i-1,j] == 0:
                    number += 1

                inside_fractal[i,j] = number
    
    inside_fractal_flat = inside_fractal.flatten()

    plt.figure(figsize = (10,10))
    plt.imshow(inside_fractal)
    plt.colorbar()
    plt.show()
    
    return inside_fractal, inside_fractal_flat



In [1]:
def plot_vector(vec, boolean_grid,x_grid,y_grid):
    '''
    Reshapes the eigenvectors due to the flattened grid earlier
    '''

    vec_reshaped = vec.reshape((len(x_grid),len(y_grid))) + boolean_grid*00.1 

    #Legge til grid på slutten for å "løfte" heile fraktalen opp så plotta blir finere.
   
    return vec_reshaped

In [17]:
def find_eigenvalue(grid_flat):
    '''
    Function that finds the eigenvalues and eigenvectores of the fractal by using the 
    central finite differences method
    '''

    n = len(grid_flat) #The size of the un-flattened matrix squared.

    N = int(np.sqrt(n)) #The size of the un-flattened matrix.

    #Matrix in the 5-point central finite differences method.
    Laplace_init = scipy.sparse.diags([-1, -1, 4, -1, -1], [-N, -1, 0, 1, N], shape=(n, n))
   
    #Matrix that contains only the value 0, except for at the diagonal at which grid_flat is placed.
    modification = scipy.sparse.diags(grid_flat, 0, shape=(n, n))

    #The modified Laplace matris. When these are multiplied we only consider relevant values within the fractal.
    Laplace = modification @ Laplace_init
    LaMemory = Laplace.data.nbytes
    return Laplace




In [ ]:
def higher_order(grid_flat):
    '''
    Parameters:
        grid_flat = One long array containing all rows in a boolean grid after one another.

    Output:
        40 eigenvalues and corresponding eigenvectors for grid_flat.
    
    Function that finds the eigenvalues and eigenvectores of the fractal by using a higher central finite differences method.
    '''

    n = len(grid_flat) #The size of the un-flattened matrix squared.

    N = int(np.sqrt(n)) #The size of the un-flattened matrix.

    #Matrix in the higher central finite differences method.
    Laplace_init = scipy.sparse.diags([1, -16, 1, -16, 60, -16, 1, -16, 1], [-2*N, -N, -2, -1, 0, 1, 2, N, 2*N], shape=(n, n))
   
    #Matrix that contains only the value 0, except for at the diagonal at which grid_flat is placed.
    modification = scipy.sparse.diags(grid_flat, 0, shape=(n, n))

    #The modified Laplace matris. When these are multiplied we only consider relevant values within the fractal.
    Laplace = modification @ Laplace_init
    
    return scipy.sparse.linalg.eigsh(Laplace, 40, which = 'SA')

In [ ]:
def generateModifiedLaplace(boolean):
    '''
    Generates a modified Laplace due to boundary conditions
    '''

    length_border = int(boolean.sum())
    
    border_information = np.zeros((length_border,5))
   
    inside_fractal_with_border = scipy.ndimage.binary_fill_holes(boolean).astype(int)
    
    inside_fractal = inside_fractal_with_border-boolean
    
    inside_fractal_flat = inside_fractal.flatten()
    
    for i in range(inside_fractal.shape[0]):
        for j in range(inside_fractal.shape[1]):

            if inside_fractal[i,j] == 1:
                
                if inside_fractal[i,j+1] == 0:
                    border_information[i+j, 0] = 1

                if inside_fractal[i+1,j] == 0:
                    border_information[i+j, 1] = 1

                if inside_fractal[i-1,j] == 0:
                    border_information[i+j, 2] = 1

                if inside_fractal[i,j-1] == 0:
                    border_information[i+j, 3] = 1

                border_information[i+j, 4] = i+j
                

    n = len(inside_fractal_flat) #The size of the un-flattened matrix squared.

    N = int(np.sqrt(n)) #The size of the un-flattened matrix.

    #Matrix in the higher central finite differences method.
    Laplace_init = scipy.sparse.diags([-1, -1, 4, -1, -1], [-N, -1, 0, 1, N], shape=(n, n)).tocsr() 
    
    #Laplace_init = scipy.sparse.csr_matrix(Laplace_init)

    for k in range(len(border_information)):

        column = int(border_information[k,4])
        
        if border_information[k,0] == 1 and border_information[k,4] >= N: 
            Laplace_init[column-6,column] -= 1 #The column in interest

        if border_information[k,1] == 1:
            Laplace_init[column-1,column] -= 1

        if border_information[k,2] == 1:
            Laplace_init[column+1,column] -= 1

        if border_information[k,3] == 1 and border_information[i,4] <= (n-N): 
            Laplace_init[column+1,column] -= 1

    
    modification = scipy.sparse.diags(inside_fractal_flat, 0, shape=(n, n))

    #The modified Laplace matrix.
    Laplace = modification @ Laplace_init

    return Laplace

In [19]:
def clampedThinPlate(boolean, num_eig):
    '''
    Finding the eigenvalues and eigenvectors of the clamped thin plate. 
    '''

    Laplace = generateModifiedLaplace(boolean)
    Laplace2 = Laplace @ Laplace


    return scipy.sparse.linalg.eigsh(Laplace2, num_eig, which = 'SA')